In [2]:
import json
import math

with open("roadnet_4X4.json") as f:
    roadnet = json.load(f)

with open("hangzhou_4x4_gudang_18041610_1h.json") as f:
    flows = json.load(f)

nodes = []
positions = {}
for inter in roadnet["intersections"]:
    nid = inter["id"]
    x = float(inter["point"]["x"])
    y = float(inter["point"]["y"])
    nodes.append(f'<node id="{nid}" x="{x}" y="{y}" type="traffic_light"/>')
    positions[nid] = (x, y)

edges = []
for road in roadnet["roads"]:
    if not road["startIntersection"] or not road["endIntersection"]:
        continue
    rid = road["id"]
    from_node = road["startIntersection"]
    to_node = road["endIntersection"]
    num_lanes = len(road["lanes"])
    speed = road["lanes"][0]["maxSpeed"] if road["lanes"] else 13.9
    fx, fy = positions[from_node]
    tx, ty = positions[to_node]
    length = round(math.hypot(tx - fx, ty - fy), 2)
    lane_strs = [f'<lane id="{rid}_{i}" index="{i}" speed="{speed}" length="{length}"/>' for i in range(num_lanes)]
    edges.append(f'<edge id="{rid}" from="{from_node}" to="{to_node}">\n' + "\n".join(lane_strs) + "\n</edge>")

vehicles = []
for i, flow in enumerate(flows):
    route = " ".join(flow["route"])
    depart = flow.get("startTime", i * 5)
    vehicles.append(f'<vehicle id="veh{i}" type="car" depart="{depart}"><route edges="{route}"/></vehicle>')

with open("hangzhou_from_json.nodes.xml", "w") as f:
    f.write("<nodes>\n" + "\n".join(nodes) + "\n</nodes>")

with open("hangzhou_from_json.edges.xml", "w") as f:
    f.write("<edges>\n" + "\n".join(edges) + "\n</edges>")

with open("hangzhou_from_json.rou.xml", "w") as f:
    f.write("<routes>\n")
    f.write('<vType id="car" accel="2.0" decel="4.5" length="5" minGap="2.5" maxSpeed="16.67"/>\n')
    f.write("\n".join(vehicles))
    f.write("\n</routes>")

with open("hangzhou_from_json.netccfg", "w") as f:
    f.write("nodes = hangzhou_from_json.nodes.xml\n")
    f.write("edges = hangzhou_from_json.edges.xml\n")
    f.write("output-file = hangzhou_from_json.net.xml\n")
    f.write("tls.guess = true\n")
    f.write("tls.join = true\n")

print("✅ All files written.")
print("➡️  Now run: netconvert -c hangzhou_from_json.netccfg")


✅ All files written.
➡️  Now run: netconvert -c hangzhou_from_json.netccfg


In [3]:
import json

with open("roadnet_4X4.json") as f:
    roadnet = json.load(f)

edges_xml = ['<edges>']

for road in roadnet["roads"]:
    if not road["startIntersection"] or not road["endIntersection"]:
        continue

    edge_id = road["id"]
    from_node = road["startIntersection"]
    to_node = road["endIntersection"]

    for i, lane in enumerate(road["lanes"]):
        speed = lane.get("maxSpeed", 13.89)  # 50 km/h default
        length = lane.get("length", 300)
        lane_id = f"{edge_id}_{i}"
        edges_xml.append(
            f'  <edge id="{lane_id}" from="{from_node}" to="{to_node}" priority="1">\n'
            f'    <lane index="{i}" speed="{speed}" length="{length}"/>\n'
            f'  </edge>'
        )

edges_xml.append("</edges>")

with open("hangzhou_from_json.edges.xml", "w") as f:
    f.write("\n".join(edges_xml))

print("✅ TAPAS-style edges.xml saved as hangzhou_from_json.edges.xml")


✅ TAPAS-style edges.xml saved as hangzhou_from_json.edges.xml


In [5]:
import json
from xml.etree.ElementTree import Element, SubElement, ElementTree

# === Load JSON Files ===
with open("roadnet_4X4.json") as f:
    roadnet = json.load(f)
with open("hangzhou_4x4_gudang_18041610_1h.json") as f:
    flows = json.load(f)

# === Create nodes.xml ===
nodes_root = Element("nodes")
for inter in roadnet["intersections"]:
    SubElement(
        nodes_root,
        "node",
        {
            "id": inter["id"],
            "x": str(inter["point"]["x"]),
            "y": str(inter["point"]["y"]),
            "type": "traffic_light" if not inter["virtual"] else "priority"
        },
    )
ElementTree(nodes_root).write("nodes.xml", encoding="utf-8", xml_declaration=True)

# === Create edges.xml ===
edges_root = Element("edges")
for road in roadnet["roads"]:
    from_node = road["startIntersection"]
    to_node = road["endIntersection"]
    if from_node is None or to_node is None:
        continue  # Skip malformed roads

    edge_elem = SubElement(
        edges_root,
        "edge",
        {
            "id": road["id"],
            "from": from_node,
            "to": to_node,
        },
    )
    for i, lane in enumerate(road["lanes"]):
        SubElement(
            edge_elem,
            "lane",
            {
                "index": str(i),
                "speed": str(lane.get("maxSpeed", 11.11)),
                "length": str(lane.get("length", 300)),
            },
        )
ElementTree(edges_root).write("edges.xml", encoding="utf-8", xml_declaration=True)

# === Create hangzhou.rou.xml ===
routes_root = Element("routes")
SubElement(
    routes_root,
    "vType",
    {
        "id": "car",
        "accel": "2.0",
        "decel": "4.5",
        "length": "5.0",
        "minGap": "2.5",
        "maxSpeed": "16.67",
    },
)

for i, flow in enumerate(flows):
    route_edges = " ".join(flow["route"])
    vehicle = SubElement(
        routes_root,
        "vehicle",
        {
            "id": f"veh{i}",
            "type": "car",
            "depart": str(flow.get("startTime", i * 5)),
        },
    )
    SubElement(vehicle, "route", {"edges": route_edges})

ElementTree(routes_root).write("hangzhou.rou.xml", encoding="utf-8", xml_declaration=True)

print("✅ Done! Generated:")
print("  • nodes.xml")
print("  • edges.xml")
print("  • hangzhou.rou.xml")
print("\n➡️ Now run this to build your .net.xml:")
print("   netconvert -c netconvert.cfg --tls.guess 1 --tls.join true")


✅ Done! Generated:
  • nodes.xml
  • edges.xml
  • hangzhou.rou.xml

➡️ Now run this to build your .net.xml:
   netconvert -c netconvert.cfg --tls.guess 1 --tls.join true
